In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn import datasets

## Простая линейная регрессия

Пусть $y$ - целевая переменная

$$y=\beta_0+\beta_1 \cdot x + \epsilon$$ 

Подгоняем модель вида $$y=\beta_0+\beta_1 \cdot x$$ 

### Допущения

1. Вид зависимости $y=\beta_0+\beta_1 \cdot x + \epsilon$

2. Независимость наблюдений друг от друга

3. $\mathbb{D}\epsilon = \sigma^2$ - дисперсия ошибок постоянна (гомоскедастичность) 

4. $\epsilon \sim N(0,\sigma^2)$


# Проблемы при использовании

1. Выбросы в данных

2. Коррелированность в признаках

3. Неадекватность выбора линейной модели 

4. Интерпретация причинно-следственных связей

### Метод наименьших квадратов
Минимизируюя сумму квадратов ошибок

$$RSS(\beta_0,\beta_1) = \sum_{t=1}^n(y-(\beta_0+\beta_1 \cdot x))^2$$

найдем оценки параметров $\beta_0$ и $\beta_1$. Вычислим частные производный и, приравнивая их нулю, получим

$$\frac{\partial RSS(\beta_0,\beta_1)}{\partial{\beta_0}}= 0$$

$$\frac{\partial RSS(\beta_0,\beta_1)}{\partial{\beta_1}}= 0$$

получим

$$\widehat{\beta}_1=\frac{\sum_{t=1}^n(y-\overline y)(x-\overline x)}{\sum_{t=1}^n(x-\overline x)^2}$$

$$\widehat{\beta_0}=\overline{y} -\beta_1x$$

In [ ]:
salary = pd.read_csv('SalaryData.csv')
salary = salary.rename(columns={'Experience Years': 'Experience'})
y = salary['Salary']
x = salary[['Experience']]
salary

In [ ]:
plt.plot(x,y)

In [ ]:
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

In [ ]:
sm.add_constant
model = ols("Salary ~ Experience ", data=salary)
result = model.fit()

In [ ]:
print(result.summary())

In [ ]:
print("Parameters: ", result.params)
print("Standard errors: ", result.bse)
print("Predicted values: ", result.predict())

### **Разбор вывода**

**Dep. Variable**: Salary - то, что мы пытаемся объяснить

**Number of observations** - кол-во наблюдений

**Df Residuals** - кол-во наблюдений - (кол-во переменных в модели + 1 (константа))

**Df Model** - (кол-во переменных в модели)

**Intercept** = $\beta_0$ из формулы модели

**Experience** = $\beta_1$ из формулы модели (приращение на единицу роста)

**Standard Error of Parameters** - среднеквадратичное отклонение оценок параметров

**t-statistics** является результатом деления $coef/std~err$. Статистика для проверки нулевой гипотезы о равенстве соответствующего коэффициента нулю.

**P>|t|** - p-value для проверки описанной выше гипотезы

**[0.025 0.975]** - доверительные интервалы для значений параметров

**R-Squared (коэффициент детерминации)** - показывает долю объясненной дисперсии целевой переменной с помощью предикторов. Изменяется от 0 до 1, 1 - полностью объясненная модель.

**Adjusted R-Squared (cкорректированный коэффициент детерминации)** - это модифицированная версия коэффициента детерминации, которая корректирует количество независимых переменных в модели, обеспечивая более точную оценку объяснительной силы модели при сравнении моделей с разным количеством переменных. Если скорректированный R-квадрат уменьшается при добавлении дополнительных переменных, это говорит о том, что дополнительные переменные не вносят существенного вклада в модель и могут быть опущены.

**F-статистика и Prob(F-статистика)** F-статистика используется для проверки общей значимости модели. Нулевая гипотеза заключается в том, что все коэффициенты (кроме константы) равны нулю, что означает, что модель не объясняет никакой дисперсии в целевой переменной. Значение p, связанное с F-статистикой, указывает на вероятность наблюдения F-статистики (или более экстремального значения), если бы нулевая гипотеза была верна. Небольшое значение p (обычно менее 0,05) указывает на то, что модель статистически значима, что означает, что по крайней мере одна из независимых переменных оказывает значительное влияние на целевую переменную.

**AIC:** - информационный критерий Акаике, численное значение само по себе ничего не дает, используется для сравнения моделей - чем меньше, тем лучше. Формула $AIC = 2*k -2*log~likehood$

**BIC** - Байесовский информационный критерий ($BIC = ln(n)*k -2*log~likehood$), аналогично чем меньше - тем лучше

**Omnibus:** Тест оценивает совместную нормальность остатков. Более высокое значение предполагает отклонение от нормальности.

**Prob(Omnibus)** - это p-value указывает на вероятность наблюдения тестовой статистики при нулевой гипотезе нормальности. Значение выше установленной границы (0.05 к примеру) предполагает, что мы не отвергаем нулевую гипотезу, подразумевая, что остатки могут быть распределены нормально.

**Skew (ассиметрия)**:измеряет асимметрию распределения остатков. Значение асимметрии, близкое к нулю, указывает на симметричное распределение, тогда как положительные или отрицательные значения указывают на правый или левый перекос соответственно.

**Kurtosis (эксцесс)**: Эксцесс измеряет «хвостость» распределения. Значение эксцесса 3 указывает на нормальное распределение, тогда как значения выше или ниже предполагают более тяжелые или более легкие хвосты соответственно.

**Тест Жака-Бера (JB):** Тест Жака-Бера — это еще один тест на нормальность, который оценивает, соответствуют ли асимметрия и эксцесс выборки показателям нормального распределения.

**Prob(JB):** это p-value оценивает нулевую гипотезу нормальности. Значение выше установленной границы (0.05 к примеру) указывает на то, что мы не отвергаем нулевую гипотезу.

**Durbin-Watson:** Эта статистика проверяет наличие автокорреляции в остатках регрессионного анализа. Значения, близкие к 2, предполагают отсутствие автокорреляции, тогда как значения меньше 1 или больше 3 указывают на положительную или отрицательную автокорреляцию соответственно.

**Cond. No.** оценивает мультиколлинеарность, где значения выше 30 указывают на потенциальные проблемы мультиколлинеарности среди независимых переменных.


In [ ]:
fig = sm.graphics.plot_partregress_grid(result)

In [ ]:
fig = sm.graphics.plot_regress_exog(result, "Experience")

In [ ]:
fig = sm.graphics.plot_fit(result, "Experience")

In [ ]:
salary = pd.read_csv('SalaryData.csv')
salary = salary.rename(columns={'Experience Years': 'Experience'})
y = salary['Salary']
x = salary[['Experience']]
y.iloc[15] += 10**5
y.iloc[16] += 10**5
y.iloc[18] += 10**5
salary


In [ ]:
plt.plot(x,y)

In [ ]:
print(salary)

In [ ]:
salary.loc[15,'Salary'] += 10**5
salary.loc[16,'Salary'] += 10**5
salary.loc[17,'Salary'] += 10**5
salary.loc[18,'Salary'] += 10**5

salary.loc[35,'Salary'] /= 10
salary.loc[36,'Salary'] /= 10
salary.loc[37,'Salary'] /= 10
salary.loc[38,'Salary'] /= 10


sm.add_constant
model = ols("Salary ~ Experience ", data=salary)
result = model.fit()
print(result.summary())

In [ ]:
fig = sm.graphics.plot_regress_exog(result, "Experience")

In [ ]:
fig = sm.graphics.plot_fit(result, "Experience")

# Множественная линейная регрессия

Пусть $y$ - целевая переменная

$$y=\beta_0+\beta_1 \cdot x_1 + \beta_2 \cdot x_2 + \dots + \beta_k \cdot x_k +\epsilon$$ 

Подгоняем модель вида $$y=\beta_0+\beta_1 \cdot x_1 + \beta_2 \cdot x_2 + \dots + \beta_k \cdot x_k$$ 


Общим в этих примерах является то, что они линейны по отношению к неизвестным параметрам 
$\beta_0,\beta_1,...,\beta_{p}$, поэтому модель называют линейной регрессионной моделью.

Наиболее популярным методом получения оценок параметров $\beta_0,\beta_1,...,\beta_{p}$. является так называемый метод наименьших квадратов. 

Метод заключается в минимизации суммы квадратов ошибок $\epsilon^{T}\epsilon=\sum_{i}\epsilon_{i}^2$ относительно вектора параметров $\beta=[\beta_0,\beta_1₁,...,\beta_p]$
    
Представим $\epsilon^{T}\epsilon$ в виде

$$\epsilon^{T}\epsilon=(Y-X\beta)^{T}(Y-X\beta)=Y^{T}Y-2\beta^{T}X^{T}Y+\beta^{T}X^{T}X\beta$$

Продифференцируем $\epsilon^{T}\epsilon$ по $\beta$. Пpиравняем полученную производную нулю получим

$$-2X^{T}Y+2X^{T}X\beta=0$$

или

$$X^{T}X\beta=X^{T}Y$$ 

Последние уравнения называют нормальными уравнениями. Решение этого уравнения вектор $\tilde\beta$ называют оценкой наименьших квадратов. 

Простой проверкой нетрудно убедиться, что $\tilde\beta$ доставляет  минимум для $\epsilon^{T}\epsilon.\epsilon^{T}\epsilon$ обозначают как RSS.


Вектор $\widetilde{Y}=X\tilde\beta=\tilde\beta_0+\tilde\beta_1x_{t,1}+...\tilde\beta_{p}x_{t,p}$ называют регрессией, а вектор 
$ e = Y-\widetilde Y$ называют остатками.

$e=Y-\widetilde{Y}=Y-X\tilde\beta$

Отметим, что при известных из линейной алгебры условиях на матрицу $X^{T}X$ оценки $\tilde\beta$ и ошибки $e$ будут единственны.    

### Свойства оценок наименьших квадратов

1. Предположим, что вектор ошибок имеем нулевое математическое ожидание $E[\epsilon]=0$. Тогда 
$E[\tilde\beta]=(X^{T}X)^{-1}X^{T}E[Y]=(X^{T}X)^{-1}X^{T}X\beta=\beta$

Таким образом $\tilde\beta$ являются несмещеными оценками вектора $\beta$.

2. Предположим, что все $\epsilon_{i}   i=1,...,n$ некоррелированы и имеют одинаковую дисперсию, т.е. $D[\epsilon]=\sigma^2I_{n}$ тогда

$D[Y]=D[Y-X\tilde\beta]=D[\epsilon]=\sigma^2I_{n}$

И

$D[\tilde\beta]=D[(X^{T}X)^{-1}X^{T}Y]=(X^{T}X)^{-1}X^{T}D[Y]X(X^{T}X)^{-1}=\sigma^2(X^{T}X)^{-1}X^{T}X(X^{T}X)^{-1}=\sigma^2(X^{T}X)^{-1}$

Возникает вопрос почему именно $\tilde\beta$ наиболее популярна при оценке параметров в линейной регрессионной модели.
Ответ на этот вопрос дает следующая теорема

**Теорема.**

Пусть $θ$ - оценка наименьших квадратов вектора $θ=X \tilde\beta$. Тогда в классе всех других линейных несмещенных оценок линейной комбинации $c^{T}θ$, оценка $c'θ$ является единственной оценкой с минимальной дисперсией.

До сих пор мы не делали никаких предположений о распределении. Если предоположить независимость и нормальность распределения ошибок $\epsilon_t\sim N(0,\sigma^2)$ то Rao в 1974 году доказал, что $c^{T}\tilde\beta$ имеет минимальную дисперсию не только в классе всех линейных несмещенных оценок, но и в классе всех несмещенных оценок (эффективность). В этом случае также оценка максимального правдоподобия совпадает с оценкой наименьших квадратов. В случае не нормальности рапределения ошибок условия при которых оценка наименьших будет является ассимптотически эффективной найдены были Cox в 1968 году.

Дополнительно будем предполагать нормальность  и независимость ошибок $\epsilon_{i} \sim N(0,\sigma^2)\  или\  \epsilon∼N(0,I_{n}\sigma^2)$

**Теорема 2.** Если $Y\sim N(X\beta,I_{n}\sigma^2)$ и матрица $X$ размера $(n⊗p)$   имеет ранг $p$, тогда

A) $\tilde\beta \sim N(\beta,\sigma^2(X^{T}X)^{-1})$

B) $(\tilde\beta-\beta)^{T}X(\tilde \beta-\beta)/\sigma^2\sim\chi^2_p$

C) $\tilde\beta$ не зависит от  $s^2$ как случайные величины

D) $RSS/\sigma^2=(n-p)s^2/\sigma^2∼\chi_{n-p}²$    


In [ ]:
salary = pd.read_csv('multipleSalaryData.csv')
salary

In [ ]:
sm.add_constant
model = ols("income ~ experience", data=salary).fit()
print(model.summary())

In [ ]:
model = ols("income ~ age", data=salary).fit()
print(model.summary())

In [ ]:
sm.add_constant
model = ols("income ~ experience + age ", data=salary).fit()
print(model.summary())

In [ ]:
sm.add_constant
model = ols("income ~ experience + age - 1", data=salary).fit()
print(model.summary())

In [ ]:
fig = sm.graphics.plot_regress_exog(model,"experience")

In [ ]:
fig = sm.graphics.plot_fit(model, "experience")

In [ ]:
fig = sm.graphics.plot_fit(model, "age")

In [ ]:
fig = sm.graphics.plot_regress_exog(model,"age")

In [ ]:
income = pd.read_csv('multipleSalaryData.csv')
X1, y1 = income.iloc[:, :-1].values, income.iloc[:, -1].values
X1_scaled = scale(X1)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)
X1_train_s, X1_test_s, y1_train, y1_test = train_test_split(X1_scaled, y1, random_state=0)
sk_linear_regression = LinearRegression()
sk_linear_regression.fit(X1_train, y1_train)

sk_lr_pred_res = sk_linear_regression.predict(X1_test)
sk_lr_r2 = r2_score(y1_test, sk_lr_pred_res)
sk_lr_mape = mean_absolute_percentage_error(y1_test, sk_lr_pred_res)

print(f'Scikit-learn Linear regression R2 score: {sk_lr_r2}')
print(f'Scikit-learn Linear regression MAPE: {sk_lr_mape}', '\n')

print(f'weights: {sk_linear_regression.intercept_, *sk_linear_regression.coef_}')
print(f'prediction: {sk_lr_pred_res}', '\n')

sk_linear_regression.fit(X1_train_s, y1_train)
print(f'scaled weights: {sk_linear_regression.intercept_, *sk_linear_regression.coef_}')

In [ ]:
feature1, feature2 = X1[:, 0], X1[:, 1]
X1_linspace = np.linspace(feature1.min(), feature1.max())
X2_linspace = np.linspace(feature2.min(), feature2.max())
X1_surface, X2_surface = np.meshgrid(X1_linspace, X2_linspace)
X_surfaces = np.array([X1_surface.ravel(), X2_surface.ravel()]).T

sk_linear_regression = LinearRegression()
sk_linear_regression.fit(X1_train, y1_train)
y_surface = sk_linear_regression.predict(X_surfaces).reshape(X1_surface.shape)

fig = plt.figure(figsize=(9, 7))
ax = plt.axes(projection='3d')
ax.scatter(feature1, feature2, y1, color='red', marker='o')
ax.plot_surface(X1_surface, X2_surface, y_surface, color='black', alpha=0.6)
plt.title('Fitted linear regression surface')
ax.set_xlabel('Age')
ax.set_ylabel('Experience')
ax.set_zlabel('Income')
ax.view_init(20, 10)
plt.show()

## Полиномиальная регрессия

Важным случаем является подгонка полинома, то есть
Подгоняем модель вида $$y=\beta_0+\beta_1 \cdot x + \beta_2 \cdot x^2 + \dots + \beta_k \cdot x^k$$  
С помощью МНК (метод наименьших квадратов) хотим найти оценки для всех коэффициентов. Нужно минимизировать
$$S = \sum\limits_{i=1}^n (y_i - \hat{y_i})^2$$
$$S = \sum\limits_{i=1}^n (y_i - \sum\limits_{m=0}^k \beta_m x_i^m)^2$$
Приравняем все частные производные к нулю
$$S^{'}_{\beta_m} = 2\sum\limits_{i=1}^n x_i^m(\sum\limits_{m=0}^k \beta_m x_i^m - y_i) = 0$$
Раскроем внутреннюю сумму и перенесем y в правую часть
$$\sum\limits_{i=1}^n x_i^m (\beta_0 + \beta_1 x_i + ... + \beta_k x_i^k) = \sum\limits_{i=1}^n x_i^m y_i$$
Разделим обе части каждого такого уравнения на n и запишем в виде системы
$$\begin{cases} \beta_0 + \beta_1\overline{x} + ... \beta_k\overline{x^k} = \overline{y}\\ \beta_0\overline{x} + \beta_1\overline{x^2} + ... \beta_k\overline{x^{k+1}} = \overline{xy} \\ ... \\ \beta_0\overline{x^k} + \beta_1\overline{x^{k+1}} + ... \beta_k\overline{x^{2k}} = \overline{x^ky} \end{cases}$$

Решением этой системы будет вектор из коэффициентов, на которых строится модель

In [ ]:
data = pd.read_csv('Ice_cream selling data.csv')
data

In [ ]:
plt.scatter(data['Temperature (°C)'],data['Ice Cream Sales (units)'])
plt.show()

In [ ]:
#y = data['Ice Cream Sales (units)']
#x = data['Temperature (°C)']
data = data.rename(columns = {'Ice Cream Sales (units)': 'ice','Temperature (°C)': 't'})
sm.add_constant
model = ols("ice ~ t", data=data).fit()
print(model.summary())

In [ ]:
data

In [ ]:
fig = sm.graphics.plot_regress_exog(model, 't')

In [ ]:
data = pd.read_csv('Ice_cream selling data.csv')
data = data.rename(columns = {'Ice Cream Sales (units)': 'ice','Temperature (°C)': 't'})
data['t'] = data['t']**2
sm.add_constant
model = ols("ice ~ t", data=data).fit()
print(model.summary())

In [ ]:
fig = sm.graphics.plot_regress_exog(model, 't')

In [ ]:
ice_cream = pd.read_csv('Ice_cream selling data.csv')
X2, y2 = ice_cream.iloc[:, :-1], ice_cream.iloc[:, -1]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=0)
print(ice_cream.head())
feature_name, target_name = ice_cream.columns
poly_features = PolynomialFeatures(degree=3, include_bias=True)
X2_poly = poly_features.fit_transform(X2)
X2_poly_train, X2_poly_test = train_test_split(X2_poly, random_state=0)

sk_linear_regression = LinearRegression()
sk_linear_regression.fit(X2_train, y2_train)
sk_lr_pred_res = sk_linear_regression.predict(X2_test)
sk_lr_pred_all_data_res = sk_linear_regression.predict(X2)

sk_polynomial_regression = LinearRegression()
sk_polynomial_regression.fit(X2_poly_train, y2_train)
sk_poly_lr_pred_res = sk_polynomial_regression.predict(X2_poly_test)
sk_poly_lr_pred_all_data_res = sk_polynomial_regression.predict(X2_poly)

linear_regression_r2 = r2_score(y2_test, sk_lr_pred_res)
polynomial_regression_r2 = r2_score(y2_test, sk_poly_lr_pred_res)



linear_regression_mse = mean_squared_error(y2_test, sk_lr_pred_res)
polynomial_regression_mse = mean_squared_error(y2_test, sk_poly_lr_pred_res)

linear_regression_mape = mean_absolute_percentage_error(y2_test, sk_lr_pred_res)
polynomial_regression_mape = mean_absolute_percentage_error(y2_test, sk_poly_lr_pred_res)

print(f'R2 score (Linear regression): {linear_regression_r2}')
print(f'R2 score (Polynomial regression): {polynomial_regression_r2}', '\n')

print(f'MSE (Linear regression): {linear_regression_mse}')
print(f'MSE (Polynomial regression): {polynomial_regression_mse}', '\n')

print(f'MAPE (Linear regression): {linear_regression_mape}')
print(f'MAPE (Polynomial regression): {polynomial_regression_mape}')

plt.scatter(X2, y2)
plt.plot(X2, sk_lr_pred_all_data_res, color='darkorange', label='Linear Regression')
plt.plot(X2, sk_poly_lr_pred_all_data_res, color='green', label='Polynomial Regression')
plt.title('Polynomial vs Linear regression')
plt.xlabel(feature_name)
plt.ylabel(target_name)
plt.legend()
plt.show()


# Регуляризация в линейной регрессии

$$J_{LASSO} = \sum_i (y_i - \hat{y})^2 + \lambda ||w||$$

$$J_{RIDGE} = \sum_i (y_i - \hat{y})^2 + \lambda w^2$$

In [ ]:
x, y = datasets.load_diabetes(as_frame=True, return_X_y=True)
x = pd.DataFrame(x)
sns.heatmap(x.corr(),annot=True,fmt=".2f", linewidth=.5)
plt.show()

In [ ]:
bmi = x[['bmi']]
# Импорт класса LinearRegression из модуля linear_model пакета scikit-learn
from sklearn.linear_model import LinearRegression

# Создание экземпляра класса LinearRegression
simple_lr = LinearRegression()

# Обучение модели
simple_lr.fit(bmi, y)

# Прогнозирование целевой переменной и сохранение результата в predicted_y
predicted_y = simple_lr.predict(bmi)

# Построение линии регрессии на точечной диаграмме
plt.figure(figsize=(10, 6))
plt.scatter(bmi, y)
plt.plot(bmi, predicted_y, c = 'r')
plt.title('Scatter plot and a Simple Linear Regression Model')
plt.ylabel("y")
plt.xlabel("bmi")
plt.show()

In [ ]:
# Импорт функции cross_val_score из модуля model_selection scikit-learn
from sklearn.model_selection import cross_val_score

# Сохранение десяти метрик в переменной mse
mse = cross_val_score(simple_lr,
                      bmi,
                      y,
                      scoring='neg_mean_squared_error',
                      cv=10)

# Получение среднего значения для оценки качества модели
mse.mean()
# -3906.9189901068407

In [ ]:
# обучение модели на всех признаках
multiple_lr = LinearRegression().fit(x, y)

# Сохраняем массив с результатами оценки качества
mse = cross_val_score(multiple_lr,
                     x,
                     y,
                     scoring='neg_mean_squared_error',
                     cv=10)

# Получение среднего значения метрик для оценки качества модели
mse.mean()
# -3000.3902901608426

# Сохранение массива, содержащего все десять коэффициентов
multiple_lr_coeffs = multiple_lr.coef_
multiple_lr_coeffs
feature_names = x.columns

# Используем matplotlib для построения графика
plt.figure(figsize=(10, 6))
plt.plot(range(len(multiple_lr_coeffs)), multiple_lr_coeffs)
plt.axhline(0, color = 'r', linestyle = 'solid')
plt.xticks(range(len(feature_names)), feature_names, rotation = 50)
plt.title("Coefficients for Multiple Linear Regression")
plt.ylabel("coefficients")
plt.xlabel("features")
plt.show()

# Ridge регрессия

In [ ]:
# Импорт класса Ridge из модуля linear_model scikit-learn
from sklearn.linear_model import Ridge

# Импорт класса GridSearchCV из модуля model_selection scikit-learn
from sklearn.model_selection import GridSearchCV

# Создание словаря, содержащего потенциальные значения альфа
alpha_values = {'alpha': [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.08, 1, 2, 3, 5, 8, 10, 20, 50, 100]}

# Передача в GridSearchCV Ridge-модели, потенциальных альфа-значений,
# метрики качества
ridge = GridSearchCV(Ridge(),
                     alpha_values,
                     scoring='neg_mean_squared_error',
                     cv=10)

# обучение модели 
print('Лучшее значение alpha:', ridge.fit(x,y).best_params_)

# Вывод среднего значения neg_mean_squared_error
print('Метрика качества:', ridge.fit(x,y).best_score_)

# Лучшее значение alpha: {'alpha': 0.04}
# Метрика качества: -2997.195810600043

In [ ]:
# Создание объекта, содержащего наилучшую модель
best_ridge_model = Ridge(alpha=0.04)

# Извлечение оценок коэффициентов для всех десяти признаков
best_ridge_coeffs = best_ridge_model.fit(x, y).coef_

# Построение графика c коэффициентами для всех десяти признаков
plt.figure(figsize = (10, 6))
plt.plot(range(len(feature_names)), best_ridge_coeffs)
plt.axhline(0, color = 'r', linestyle = 'solid')
plt.xticks(range(len(feature_names)), feature_names, rotation = 50)
plt.title("Coefficient estimates from Ridge Regression")
plt.ylabel("coefficients")
plt.xlabel("features")
plt.show()


# Lasso

In [ ]:
# Импорт класса Lasso из модуля linear_model scikit-learn
from sklearn.linear_model import Lasso

# Создание словаря, содержащего потенциальные значения альфа
alpha_values = {'alpha': [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06,0.07, 0.08, 1, 2, 3, 5, 8, 10, 20, 50, 100]}

# Передача в GridSearchCV модели, потенциальных альфа-значений,
# метрики качества
lasso = GridSearchCV(Lasso(),
                     alpha_values,
                     scoring='neg_mean_squared_error',
                     cv=10)

# Обучение модели 
print('Лучшее значение alpha:', lasso.fit(x, y).best_params_)

# Вывод среднего значения neg_mean_squared_error 
print('Метрика качества:', lasso.fit(x, y).best_score_)

# Лучшее значение alpha: {'alpha': 0.06}
# Метрика качества:: -2987.4275179741567

In [ ]:
# Создание объекта, содержащего наилучшую Lasso-модель 
best_lasso_model = Lasso(alpha=0.06)

# сохранение значений коэффициентов для всех десяти признаков
best_lasso_coeffs = best_lasso_model.fit(x, y).coef_

# Построение графика значений коэффициентов для всех десяти объектов
plt.figure(figsize = (10, 6))
plt.plot(range(len(feature_names)), best_lasso_coeffs)
plt.axhline(0, color = 'r', linestyle = 'solid')
plt.xticks(range(len(feature_names)), feature_names, rotation = 50)
plt.title("Coefficient estimates from Lasso Regression")
plt.ylabel("coefficients")
plt.xlabel("features")
plt.show()